In [ ]:
LabelledPath = " " #Please provide path to labelled dataset
UnabelledPath = " " #Please provide path to unlabelled dataset

In [ ]:
import h5py
import numpy as np

Loading labelled and Unlabelled datasets

In [ ]:
f = h5py.File("LabelledPath" , "r")
H = f["H_Est"][:].transpose(0,3,1,2)
Pos = f["Pos"][:]
f.close()

In [ ]:
f = h5py.File("UnlabelledPath", 'r')
X = f['H_Est'][:24000].transpose(0,3,1,2)
f.close()

Creating a custom test set

In [ ]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H,Pos,test_size=0.05, random_state=42)

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import os
from torchvision.utils import save_image
from torch.autograd import Variable

Validation set

In [ ]:
H_Train, H_Val , Pos_Train , Pos_Val = train_test_split(H_Train,Pos_Train,test_size=0.1, random_state=99)

In [ ]:
H_Train = torch.tensor(H_Train,dtype=torch.float)
Pos_Train = torch.tensor(Pos_Train,dtype=torch.float)
H_Val = torch.tensor(H_Val,dtype=torch.float)
Pos_Val = torch.tensor(Pos_Val,dtype=torch.float)

In [ ]:
class Sammon(nn.Module):
    ''' Deep Convolutional NN for Postion estimation'''
    def __init__(self):
        super(Sammon, self).__init__()
        
        '''Convolutional Layers'''
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=5, out_channels=128, kernel_size=5, padding=2) ,
            nn.ReLU(),
            nn.AvgPool2d((1,4) ),
            nn.Conv2d(in_channels=128, out_channels=28, kernel_size=3, padding=1) ,
            nn.ReLU(),
            nn.AvgPool2d((1,4) )
        )
        
        ''' Fully Connected Layers'''
        self.Project = nn.Sequential(nn.Linear(57*56*28,1024),
                                     nn.ReLU(),
                                     nn.Linear(1024,512),
                                     nn.ReLU(),
                                     nn.Linear(512,256),
                                     nn.ReLU(),
                                     nn.Linear(256,128),
                                     nn.ReLU(),
                                     nn.Linear(128,3))
        
          
            
    def forward(self, input):
        return self.Project(self.main(input).reshape(-1,57*56*28))
    
class Alpha(nn.Module):
    ''' Scaling factor for position'''
    def __init__(self):
        super(Alpha, self).__init__()
        self.scale = nn.Parameter(torch.FloatTensor(1).fill_(1));        
            
    def forward(self, input):
        return self.scale*input

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
sammon = Sammon().to(device)
alpha = Alpha().to(device)

In [ ]:
''' Adam Optimizers'''
opt1 = optim.Adam(sammon.parameters(), lr=1e-3,weight_decay=1e-7)
opt3 = optim.Adam(alpha.parameters(), lr=1e-3,weight_decay=1e-7)
MSE  = nn.MSELoss()

In [ ]:
'''Generating pairs from unsupervised dataset'''
A = list(range(len(X)))*25
pairs = []
import random
while len(A)>1:
  t = random.randint(1,len(A)-1)
  if(A[t]!=A[0]) :
    pairs.append((A[0],A[t]))
    del A[0]
    del A[t-1]
pairs = np.array(pairs)

In [ ]:
def EucNorm(X,e=1e-6):
      return torch.sqrt(torch.norm(X)**2 + e)

In [ ]:
N_b2 = len(H_Train) #No of samples in labelled training set
N_b = len(pairs) #No of samples from unlabelled set
N_val = len(H_Val) #No of samples for validation
bs1= 64 #Batch Size for unsupervised learning
bs2 = 16 #Batch Size for supervised learning

In [ ]:
def Loss(XX, YY):
    ''' Sammon Loss Function'''
    return torch.sum((1/XX)*(torch.square(XX-YY)))

In [ ]:
''' Training the Model'''
for i in range(50):
  batch1=0
  SammonLoss = []
  ''' Unsupervised learning via sammon mapping'''
  for j in range(0,N_b,bs1):
    sammon.zero_grad()
    alpha.zero_grad()
    end = min(j+bs1,N_b)
    Xn,Xm = torch.tensor(X[pairs[j:end,0]],dtype=torch.float),  torch.tensor(X[pairs[j:end,1]],dtype=torch.float)
    Xn , Xm = Variable(Xn, requires_grad=True) , Variable(Xm, requires_grad=True)
    Xn = Xn.to(device)
    Xm = Xm.to(device)
    Yn = sammon(Xn)
    Ym = sammon(Xm)
    loss1 = EucNorm(Xn-Xm)
    loss2 = EucNorm(Yn-Ym)
    loss = Loss(loss1,alpha(loss2)[0])
    loss.backward()
    SammonLoss.append(loss.item())
    opt1.step()
    opt3.step()
    batch1+=1
    if(batch1%30 == 0):
      print("Batch",batch1,"/",(N_b//bs1)+1,"Sammon Loss :" , np.mean(SammonLoss))

  batch2=0 
  RegressorLoss = []
  ''' Supervised learning on labelled dataset'''
  for k in range(0,N_b2,bs2):
    end = min(k+bs2,N_b2)
    H1 = H_Train[k:end]
    sammon.zero_grad()
    
    H1 = Variable(H1,requires_grad=True)
    H1 = H1.to(device)

    Y1 = sammon(H1)
    loss_1 = MSE(Y1,Pos_Train[k:end].to(device)).view(-1)
    loss_1.backward()
    opt1.step()
    batch2+=1
    RegressorLoss.append(loss_1.item())
    if(batch2%20 == 0):
      print("Batch",batch2,"/",(N_b2//bs2)+1,"Regressor Loss :" , np.mean(RegressorLoss))
  print("Epoch ",i,"/",100," Sammon Loss :",np.mean(SammonLoss) ," Regressor Loss :" , np.mean(RegressorLoss))
  
    
  ''' Validation'''  
  H_Val = torch.tensor(H_Val,dtype=torch.float)
  H_Val = H_Val.to(device)
  with torch.no_grad():
    Y_Val = sammon(H_Val)
    loss_Val = MSE(Y_Val,Pos_Val.to(device)).view(-1)
  print("Val Loss :" , loss_Val.item())
  